In [4]:
%pip install torch


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
# Create a sample user-item rating matrix (rows = users, cols = items)
# Ratings are from 1-5, 0 indicates no rating

# Compute item-item cosine similarity matrix
# Formula: sim(i,j) = (ratings[:,i] · ratings[:,j]) / (||ratings[:,i]|| * ||ratings[:,j]||)

# Step 1: Calculate L2 norms of each item vector
# Step 2: Compute dot products between all pairs of items
# Step 3: Calculate denominator (outer product of norms) with epsilon to avoid division by zero
# Step 4: Compute final similarity matrix

# Example rating matrix: rows = users, cols = items
# 0 indicates no rating (unknown)
ratings = torch.tensor([
    [5.0, 4.0, 0.0, 1.0, 0.0],   # User 0
    [0.0, 2.0, 3.0, 0.0, 0.0],   # User 1
    [1.0, 0.0, 2.0, 5.0, 0.0],   # User 2
    [0.0, 0.0, 5.0, 4.0, 3.0]    # User 3
])

num_users, num_items = ratings.shape

# Compute item-item cosine similarity matrix
# Cosine sim(i,j) = (ratings[:,i] · ratings[:,j]) / (||ratings[:,i]|| * ||ratings[:,j]||)
# First, compute norms of each item vector
item_norms = torch.norm(ratings, dim=0)  # norm for each column (item)
# Compute the dot product between every pair of item columns
# ratings^T * ratings gives matrix of dot products between item vectors
dot_products = ratings.T @ ratings  # shape (num_items, num_items)
# Outer product of norms to get denominator matrix (and add a tiny epsilon to avoid division by zero)
denom = item_norms.unsqueeze(1) * item_norms.unsqueeze(0) + 1e-8
sim_matrix = dot_products / denom

# Print similarity matrix (rounded for readability)
print("Item-Item Cosine Similarity:\n", sim_matrix.round(decimals=2))
# Let's find the item most similar to item 0
item0_similarities = sim_matrix[0]
# Exclude item 0 itself by setting its similarity to -inf temporarily
item0_similarities[0] = -1.0  
most_similar_item = torch.argmax(item0_similarities).item()
print(f"Most similar item to Item 0 is Item {most_similar_item}")


Item-Item Cosine Similarity:
 tensor([[1.0000, 0.8800, 0.0600, 0.3000, 0.0000],
        [0.8800, 1.0000, 0.2200, 0.1400, 0.0000],
        [0.0600, 0.2200, 1.0000, 0.7500, 0.8100],
        [0.3000, 0.1400, 0.7500, 1.0000, 0.6200],
        [0.0000, 0.0000, 0.8100, 0.6200, 1.0000]])
Most similar item to Item 0 is Item 1


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the training data: list of (user_id, item_id, rating) tuples
# Each tuple represents a known rating from a user for an item
ratings = [
    (0, 0, 5.0), (0, 1, 4.0), (0, 3, 1.0),  # User 0's ratings
    (1, 1, 2.0), (1, 2, 3.0),               # User 1's ratings
    (2, 0, 1.0), (2, 2, 2.0), (2, 3, 5.0),  # User 2's ratings
    (3, 2, 5.0), (3, 3, 4.0), (3, 4, 3.0)   # User 3's ratings
]

# Define the dimensions of our rating matrix
num_users = 4   # users 0 through 3
num_items = 5   # items 0 through 4

# Define the number of latent factors (K)
# This is a hyperparameter that determines the dimensionality of our latent space
K = 3  # small number of latent features

class MatrixFactorization(nn.Module):
    """
    Matrix Factorization model for collaborative filtering.
    
    This model learns:
    - User embeddings: Each user is represented by a K-dimensional vector
    - Item embeddings: Each item is represented by a K-dimensional vector
    - User biases: Captures user rating tendencies
    - Item biases: Captures item rating tendencies
    
    The predicted rating is computed as: dot_product(user_embedding, item_embedding) + user_bias + item_bias
    """
    def __init__(self, num_users, num_items, num_factors):
        super(MatrixFactorization, self).__init__()
        # Create embedding layers for users and items
        self.user_emb = nn.Embedding(num_users, num_factors)
        self.item_emb = nn.Embedding(num_items, num_factors)
        
        # Initialize embeddings with small random values
        # This helps with training stability
        nn.init.normal_(self.user_emb.weight, std=0.1)
        nn.init.normal_(self.item_emb.weight, std=0.1)
        
        # Create bias terms for users and items
        # These capture individual rating tendencies
        self.user_bias = nn.Embedding(num_users, 1)
        self.item_bias = nn.Embedding(num_items, 1)
        
        # Initialize biases to zero
        nn.init.zeros_(self.user_bias.weight)
        nn.init.zeros_(self.item_bias.weight)
        
    def forward(self, user_ids, item_ids):
        """
        Forward pass of the model.
        
        Args:
            user_ids: Tensor of user IDs
            item_ids: Tensor of item IDs
            
        Returns:
            Predicted ratings for the given user-item pairs
        """
        # Get embedding vectors for users and items
        u_vecs = self.user_emb(user_ids)        # shape: [batch_size, K]
        i_vecs = self.item_emb(item_ids)        # shape: [batch_size, K]
        
        # Compute dot product for each user-item pair
        dot = (u_vecs * i_vecs).sum(dim=1)      # shape: [batch_size]
        
        # Add user and item biases
        u_bias = self.user_bias(user_ids).squeeze()  # shape: [batch_size]
        i_bias = self.item_bias(item_ids).squeeze()  # shape: [batch_size]
        
        return dot + u_bias + i_bias

# Create model instance and training components
model = MatrixFactorization(num_users, num_items, K)
loss_fn = nn.MSELoss()  # Mean Squared Error loss
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Adam optimizer with learning rate 0.01

# Prepare training data as tensors
user_tensor = torch.tensor([u for u, i, r in ratings])
item_tensor = torch.tensor([i for u, i, r in ratings])
rating_tensor = torch.tensor([r for u, i, r in ratings])

# Training loop
print("Starting training...")
model.train()  # Set model to training mode
for epoch in range(501):
    # Forward pass: predict ratings for all known pairs
    preds = model(user_tensor, item_tensor)
    loss = loss_fn(preds, rating_tensor)
    
    # Backpropagation
    optimizer.zero_grad()  # Clear previous gradients
    loss.backward()        # Compute gradients
    optimizer.step()       # Update parameters
    
    # Print progress every 100 epochs
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: MSE = {loss.item():.4f}")

# Display learned embeddings
print("\nLearned user embeddings:")
print(model.user_emb.weight.data)
print("\nLearned item embeddings:")
print(model.item_emb.weight.data)

# Generate recommendations for a specific user
def get_recommendations(user_id, k=2):
    """
    Generate top-k recommendations for a user.
    
    Args:
        user_id: ID of the user to generate recommendations for
        k: Number of recommendations to generate
        
    Returns:
        List of recommended item IDs
    """
    model.eval()  # Set model to evaluation mode
    
    # Create tensors for all possible items for this user
    user_ids = torch.tensor([user_id] * num_items)
    item_ids = torch.arange(num_items)
    
    # Get predicted ratings for all items
    pred_scores = model(user_ids, item_ids)
    
    # Mask items the user has already rated
    seen_items = {i for (u, i, r) in ratings if u == user_id}
    for i in seen_items:
        pred_scores[i] = -1e5  # Set very low score for seen items
    
    # Get top-k recommended items
    recommended_ids = torch.topk(pred_scores, k=k).indices.tolist()
    return recommended_ids

# Example: Get recommendations for user 0
user_id = 0
recommendations = get_recommendations(user_id, k=2)
print(f"\nTop 2 recommended items for user {user_id}: {recommendations}")

Starting training...
Epoch 0: MSE = 12.3051
Epoch 100: MSE = 0.4406
Epoch 200: MSE = 0.3193
Epoch 300: MSE = 0.0109
Epoch 400: MSE = 0.0000
Epoch 500: MSE = 0.0000

Learned user embeddings:
tensor([[ 1.1778, -1.1229,  0.9697],
        [-0.5731,  0.4176,  1.1482],
        [-1.2671, -0.3199,  1.4713],
        [-0.9060,  1.4882,  0.5722]])

Learned item embeddings:
tensor([[ 1.4137, -0.9943,  0.8733],
        [ 0.5803, -0.8284,  0.9683],
        [-0.8274,  1.8766,  0.3475],
        [-1.2905,  0.3359,  1.5119],
        [-0.5835,  0.6339,  0.4593]])

Top 2 recommended items for user 0: [4, 2]


In [7]:
# Example: 5 items, each with 3 content features (could be genre flags, for instance)
item_features = torch.tensor([
    [1.0, 1.0, 0.0],  # Item 0: perhaps this means it has Feature0 and Feature1
    [1.0, 0.0, 1.0],  # Item 1: has Feature0 and Feature2
    [0.0, 1.0, 1.0],  # Item 2: has Feature1 and Feature2
    [1.0, 1.0, 1.0],  # Item 3: has all three features
    [0.0, 1.0, 0.0]   # Item 4: has only Feature1
])

# Let's say our user has liked item 0 and item 2 in the past.
liked_items = [0, 2]

# Build the user profile by averaging the feature vectors of liked items (one simple strategy).
user_profile = item_features[liked_items].mean(dim=0)
print("User profile vector:", user_profile.tolist())

# Compute cosine similarity between the user profile and each item's features
norm_user = torch.norm(user_profile)
item_norms = torch.norm(item_features, dim=1)
# Compute dot product between user profile and each item feature vector
dot_scores = item_features @ user_profile  # shape: (num_items,)
cosine_sim = dot_scores / (item_norms * norm_user + 1e-8)

print("Cosine similarity of each item to the user profile:", cosine_sim.tolist())

# Exclude the items the user already liked
mask = torch.ones(item_features.size(0), dtype=torch.bool)
mask[liked_items] = False
unseen_similarity = cosine_sim * mask  # zero out seen items

# Recommend the top item based on similarity
recommended_item = torch.argmax(unseen_similarity).item()
print(f"Top recommended item for the user (content-based): Item {recommended_item}")


User profile vector: [0.5, 1.0, 0.5]
Cosine similarity of each item to the user profile: [0.8660253882408142, 0.5773502588272095, 0.8660253882408142, 0.942808985710144, 0.8164965510368347]
Top recommended item for the user (content-based): Item 3


In [8]:
import torch.nn.functional as F

# Assume we have `ratings` list and `item_features` tensor defined from before.
# (ratings: list of (user, item, rating), item_features: tensor of shape [num_items, num_features])

num_features = item_features.size(1)
hidden_dim = 8  # number of hidden units for our neural network

class HybridRecommender(nn.Module):
    def __init__(self, num_users, num_item_features, hidden_dim):
        super(HybridRecommender, self).__init__()
        # Embedding for user IDs (collaborative part)
        self.user_emb = nn.Embedding(num_users, hidden_dim)
        nn.init.normal_(self.user_emb.weight, std=0.1)
        # A simple feed-forward network for combining user embedding with item features
        self.fc1 = nn.Linear(num_item_features + hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)
    
    def forward(self, user_ids, item_feats):
        # user_ids: tensor of shape [batch_size]
        # item_feats: tensor of shape [batch_size, num_item_features]
        u_vecs = self.user_emb(user_ids)               # [batch_size, hidden_dim]
        x = torch.cat([u_vecs, item_feats], dim=1)     # combine user embedding and item features
        x = F.relu(self.fc1(x))
        x = self.fc2(x)                                # output a single score (rating)
        return x.squeeze()  # return 1D tensor (batch of scores)

# Instantiate model
hybrid_model = HybridRecommender(num_users, num_features, hidden_dim)
optimizer = optim.Adam(hybrid_model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

# Prepare data (same ratings list, but now need item feature for each example)
user_ids = torch.tensor([u for u, i, r in ratings])
item_ids = [i for u, i, r in ratings]
item_feats_tensor = torch.stack([item_features[i] for i in item_ids])
ratings_tensor = torch.tensor([r for u, i, r in ratings])

# Training loop for the hybrid model
hybrid_model.train()
for epoch in range(501):
    pred = hybrid_model(user_ids, item_feats_tensor)
    loss = loss_fn(pred, ratings_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: MSE = {loss.item():.4f}")

# Let's use the trained hybrid model to predict user 3's rating for item 0 (as an example)
hybrid_model.eval()
test_user = torch.tensor([3])
test_item_feat = item_features[0].unsqueeze(0)  # features for item 0
predicted_rating = hybrid_model(test_user, test_item_feat).item()
print(f"Predicted rating for User 3 on Item 0: {predicted_rating:.2f}")


Epoch 0: MSE = 15.3861
Epoch 100: MSE = 1.7804
Epoch 200: MSE = 1.7712
Epoch 300: MSE = 1.7652
Epoch 400: MSE = 1.7614
Epoch 500: MSE = 1.7589
Predicted rating for User 3 on Item 0: 3.76


In [15]:
class NeuralCF(nn.Module):
    def __init__(self, num_users, num_items, embed_dim, hidden_dims):
        super(NeuralCF, self).__init__()
        # Embedding layers for user and item
        self.user_emb = nn.Embedding(num_users, embed_dim)
        self.item_emb = nn.Embedding(num_items, embed_dim)
        nn.init.normal_(self.user_emb.weight, std=0.1)
        nn.init.normal_(self.item_emb.weight, std=0.1)
        # Fully connected layers for the MLP part
        # We'll create a sequential model from the list of hidden_dims
        layers = []
        input_dim = embed_dim * 2  # since we will concatenate user and item embedding
        for h in hidden_dims:
            layers.append(nn.Linear(input_dim, h))
            layers.append(nn.ReLU())
            input_dim = h
        layers.append(nn.Linear(input_dim, 1))  # final output layer
        self.mlp = nn.Sequential(*layers)
    
    def forward(self, user_ids, item_ids):
        u = self.user_emb(user_ids)
        v = self.item_emb(item_ids)
        # Concatenate user and item embedding vectors
        x = torch.cat([u, v], dim=1)
        # Pass through MLP
        out = self.mlp(x)
        return out.squeeze()  # return a scalar prediction

# Instantiate the neural CF model
ncf_model = NeuralCF(num_users, num_items, embed_dim=5, hidden_dims=[20, 10])
optimizer = optim.Adam(ncf_model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

user_tensor = torch.tensor([u for u, i, r in ratings])
item_tensor = torch.tensor([i for u, i, r in ratings])
rating_tensor = torch.tensor([r for u, i, r in ratings])

# Training loop
ncf_model.train()
for epoch in range(501):
    pred = ncf_model(user_tensor, item_tensor)
    loss = loss_fn(pred, rating_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: MSE = {loss.item():.4f}")

# Example prediction: user 0 on item 2 (which user 0 has not rated in our data)
ncf_model.eval()
test_user = torch.tensor([0])
test_item = torch.tensor([2])
score = ncf_model(test_user, test_item).item()
print(f"Neural CF predicted score for User 0 on Item 2: {score:.3f}")


Epoch 0: MSE = 11.3887
Epoch 100: MSE = 1.6547
Epoch 200: MSE = 1.6515
Epoch 300: MSE = 1.6515
Epoch 400: MSE = 1.6515
Epoch 500: MSE = 1.6515
Neural CF predicted score for User 0 on Item 2: 3.500
